In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import multiprocessing
from itertools import repeat


import matplotlib.pyplot as plt
%matplotlib inline


from pyomo.environ import *

### This notebook identifies the lower and upper bounds of the aggregate charging power and stored energy in EVs in the Chicago Metropolitan area
- First calculate the results for the base case
- Conduct the simulations under different EV hosting capacity scales, charger availability levels, EV penetration levels, and spatial allocation of EV hosting capacities

In [2]:
N_n = 7
N_t = 96
feasibleTrip = np.load('Parameters/Chicago2017_RealCapacity/pubCharger/feasible_chain.npy')
N_e_total = feasibleTrip.shape[0]

q = sparse.load_npz('Parameters/Chicago2017_RealCapacity/pubCharger/q_rec.npz').toarray()[0][feasibleTrip]

E_init = 0.5
E_terminal = 0.5-1e-3
L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1
demand_cum = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/demand_cum.npz").toarray()[feasibleTrip]

N_e = 1000
groupNum = N_e_total // N_e
N_e_last = N_e_total - N_e * groupNum


Location = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/Location.npz")[feasibleTrip]


In [3]:
mapping = pd.read_csv('TripChains/county_Mapping.csv', index_col = 0)
mappings = {}
for i in range(len(mapping)):
    mappings[mapping['nodeIndex'][i]] = mapping['countyIndex'][i]
countyIndices = mapping['countyIndex'].unique()
countyNodes = {}
for n in range(len(countyIndices)):
    countyNodes[countyIndices[n]] = n

In [4]:
boolIndex = {}
for countyInd in countyIndices:
    boolIndex[countyInd] = np.zeros((N_e_total, N_t))
    for t in range(N_t):
        boolIndex[countyInd][:, t] = Location[:, t].toarray().T[0] == countyInd


def getCountyInd(x):
    if x == -1:
        return -1
    else:
        return countyNodes[x]

vectorized_f = np.vectorize(getCountyInd)
location_countyID = vectorized_f(Location.toarray())
    

In [5]:
# groupShare = np.zeros(groupNum)
# for i in range(groupNum):
#     groupShare[i] = q[i * N_e : (i + 1) * N_e].sum()/q.sum()

# groupShare_last = q[groupNum * N_e:].sum()/q.sum()


In [7]:
'''problem for each group'''

model = ConcreteModel()

# Variables
model.y_e_UB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.y_e_LB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.E_e_UB_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals)
model.E_e_LB_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals)
model.E_e_sum_weighted = Var(range(N_e), domain=NonNegativeReals, initialize=0)

model.y_capacity = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)

model.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model.power_UB_para = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.demand_cum = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)

# model.pene_para = Param(within=NonNegativeReals, mutable=True)
model.omega = Param(range(N_e), within=NonNegativeReals, mutable=True)
# model.q = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# pene_para * q[e] * omega[e] 
model.coe_E = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model.coe_county0 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) 
model.coe_county1 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county2 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county3 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county4 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county5 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county6 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)



# Constraints for individual trip chain
model.c1 = ConstraintList()
for e in range(N_e):
    for t in range(N_t):
        model.c1.add(expr=model.y_e_UB[e, t] <= model.y_capacity[e, t])
        model.c1.add(expr=model.y_e_LB[e, t] <= model.y_capacity[e, t])
        model.c1.add(expr=model.y_capacity[e, t] <= model.omega[e] * model.power_UB_para[e, t])
        model.c1.add(expr=model.E_e_LB_omega[e, t] <= model.E_e_UB_omega[e, t])
        model.c1.add(expr=model.omega[e] * E_init - model.omega[e] * model.demand_cum[e, t] + 
                    sum(L[t, tt] * model.y_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_UB_omega[e, t])
        model.c1.add(expr=model.omega[e] * E_init - model.omega[e] * model.demand_cum[e, t] + 
                    sum(L[t, tt] * model.y_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_LB_omega[e, t])
        model.c1.add(expr=model.E_e_LB_omega[e, t] >= 0)
        model.c1.add(expr=model.E_e_UB_omega[e, t] <= model.omega[e])
    model.c1.add(expr=model.E_e_LB_omega[e, N_t-1] >= model.omega[e] * E_terminal)
    model.c1.add(expr=model.E_e_sum_weighted[e] == model.coe_E[e] * sum(model.E_e_UB_omega[e, t]-model.E_e_LB_omega[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model.c2 = ConstraintList()
for t in range(N_t):
    model.c2.add(sum(model.coe_county0[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[0])
    model.c2.add(sum(model.coe_county1[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[1])
    model.c2.add(sum(model.coe_county2[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[2])
    model.c2.add(sum(model.coe_county3[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[3])
    model.c2.add(sum(model.coe_county4[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[4])
    model.c2.add(sum(model.coe_county5[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[5])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[6])

# Objective
model.obj = Objective(expr=sum(model.E_e_sum_weighted[e] for e in range(N_e)), sense=maximize)

In [18]:
'''problem for each group'''

model_last = ConcreteModel()

# Variables
model_last.y_e_UB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.y_e_LB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.E_e_UB_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals)
model_last.E_e_LB_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals)
model_last.E_e_sum_weighted = Var(range(N_e_last), domain=NonNegativeReals, initialize=0)

model_last.y_capacity = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)

model_last.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model_last.power_UB_para = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.demand_cum = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# model_last.pene_para = Param(within=NonNegativeReals, mutable=True)
model_last.omega = Param(range(N_e_last), within=NonNegativeReals, mutable=True)
# model_last.q = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# pene_para * q[e] * omega[e] 
model_last.coe_E = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model_last.coe_county0 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) 
model_last.coe_county1 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county2 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county3 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county4 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county5 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county6 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)


# Constraints for individual trip chain
model_last.c1 = ConstraintList()
for e in range(N_e_last):
    for t in range(N_t):
        model_last.c1.add(expr=model_last.y_e_UB[e, t] <= model_last.y_capacity[e, t])
        model_last.c1.add(expr=model_last.y_e_LB[e, t] <= model_last.y_capacity[e, t])
        model_last.c1.add(expr=model_last.y_capacity[e, t] <= model_last.omega[e] * model_last.power_UB_para[e, t])
        model_last.c1.add(expr=model_last.E_e_LB_omega[e, t] <= model_last.E_e_UB_omega[e, t])
        model_last.c1.add(expr=model_last.omega[e] * E_init - model_last.omega[e] * model_last.demand_cum[e, t] + 
                    sum(L[t, tt] * model_last.y_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_UB_omega[e, t])
        model_last.c1.add(expr=model_last.omega[e] * E_init - model_last.omega[e] * model_last.demand_cum[e, t] + 
                    sum(L[t, tt] * model_last.y_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_LB_omega[e, t])
        model_last.c1.add(expr=model_last.E_e_LB_omega[e, t] >= 0)
        model_last.c1.add(expr=model_last.E_e_UB_omega[e, t] <= model_last.omega[e])
    model_last.c1.add(expr=model_last.E_e_LB_omega[e, N_t-1] >= model_last.omega[e] * E_terminal)
    model_last.c1.add(expr=model_last.E_e_sum_weighted[e] == model_last.coe_E[e] * sum(model_last.E_e_UB_omega[e, t]-model_last.E_e_LB_omega[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model_last.c2 = ConstraintList()
for t in range(N_t):
    model_last.c2.add(sum(model_last.coe_county0[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[0])
    model_last.c2.add(sum(model_last.coe_county1[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[1])
    model_last.c2.add(sum(model_last.coe_county2[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[2])
    model_last.c2.add(sum(model_last.coe_county3[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[3])
    model_last.c2.add(sum(model_last.coe_county4[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[4])
    model_last.c2.add(sum(model_last.coe_county5[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[5])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[6])

# Objective
model_last.obj = Objective(expr=sum(model_last.E_e_sum_weighted[e] for e in range(N_e_last)), sense=maximize)


In [1]:
# load real EV hosting capacity
capacity_county = pd.read_csv('geo/capacity_county_ordered.csv', index_col = 0)['EV_LOAD_CAPACITY_KW'].values/1000 / 56.5 * 1000

"""parameters for the base case in Section 3"""
peneList = [0.011]
ratio_pubCharger_list = [0.4]
capacity_scale = [1] # different EV hosting capacity
peneNum = 1
capacityNum = 1
ratioNum = 1


# """parameters for the analysis in Sections 4 and 5"""
# peneNum = 10
# capacityNum = 11
# capacity_scale = np.arange(capacityNum)*0.5 + 1 # different EV hosting capacity
# peneList = np.linspace(0.1, 1, peneNum) # different EV penetration rate
# ratio_pubCharger_list = np.linspace(0.1, 1, 10) # different availability level of public chargers
# ratioNum = len(ratio_pubCharger_list)

NameError: name 'pd' is not defined

In [ ]:
E_UB_rec = {}
E_LB_rec = {}
ell_UB_rec = {}

groupShare_matrix = sparse.load_npz(
                f"Parameters/Chicago2017_RealCapacity/pubCharger/groupShare/group.npz").toarray()

countyID_list = [31, 43, 89, 97, 111, 197, 93]

solver = SolverFactory('gurobi')
for i in range(peneNum):
    E_UB_rec[i] = {}
    E_LB_rec[i] = {}
    ell_UB_rec[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        E_UB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        E_LB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        ell_UB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        p_max_total = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()[feasibleTrip]
        power_UB_para_value = p_max_total
    

        for j in range(capacityNum):
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, scale={capacity_scale[j]}")
            omega_matrix = sparse.load_npz(
                f"Results/Chicago2017_RealCapacity/omega_ratio/omega_pene={peneList[i]:.1f}_ratio={ratio_pubCharger:.1f}.npz").toarray()
            capacity_value = capacity_county * capacity_scale[j] * (1+1e-5)
            pene_value = peneList[i]
            omega_value = omega_matrix[j]
            groupShare = groupShare_matrix[i].reshape(groupNum+1, N_n)
            coe_temp = pene_value * q
            model.pene_para = pene_value
            start_all = time.time()
            for g in range(groupNum):
                '''parameter seeting for each group'''
                for n in range(N_n):
                    model.ell_capacity[n] = capacity_value[n] * groupShare[g, n]

                for e in range(N_e):
                    model.coe_E[e] = pene_value * q[g * N_e + e]
                    model.omega[e] = omega_value[g * N_e + e]
                    for t in range(N_t):
                        model.power_UB_para[e, t] = power_UB_para_value[g * N_e + e, t]
                        model.demand_cum[e, t] = demand_cum[g * N_e + e, t]
                        model.coe_county0[e, t] = pene_value * q[g * N_e + e] * boolIndex[31][g * N_e + e, t]
                        model.coe_county1[e, t] = pene_value * q[g * N_e + e] * boolIndex[43][g * N_e + e, t]
                        model.coe_county2[e, t] = pene_value * q[g * N_e + e] * boolIndex[89][g * N_e + e, t]
                        model.coe_county3[e, t] = pene_value * q[g * N_e + e] * boolIndex[97][g * N_e + e, t]
                        model.coe_county4[e, t] = pene_value * q[g * N_e + e] * boolIndex[111][g * N_e + e, t]
                        model.coe_county5[e, t] = pene_value * q[g * N_e + e] * boolIndex[197][g * N_e + e, t]
                        model.coe_county6[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[93][g * N_e + e, t]

                '''solve the problem for each group'''
                start = time.time()
                results = solver.solve(model, warmstart=True)

                if str(results['Solver'][0]['Termination condition']) == 'optimal':
                    y_UB_result = np.array(model.y_e_UB[:, :]()).reshape(N_e, N_t)
                    y_LB_result= np.array(model.y_e_LB[:, :]()).reshape(N_e, N_t)
                    ell_UB_result = np.array(model.y_capacity[:, :]()).reshape(N_e, N_t)


                    E_UB_group = np.zeros((N_n, N_t))
                    E_LB_group = np.zeros((N_n, N_t))
                    ell_UB_group = np.zeros((N_n, N_t))
                    for e in range(N_e):
                        E_UB_county_temp = np.zeros((N_n, N_t))
                        E_LB_county_temp = np.zeros((N_n, N_t))
                        ell_UB_temp = np.zeros((N_n, N_t))
                        for n in range(N_n):
                            countyID = countyID_list[n]
                            E_UB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * y_UB_result[e] * 0.25
                            E_LB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * y_LB_result[e] * 0.25
                            ell_UB_temp[n] = boolIndex[countyID][g * N_e+e, :] * ell_UB_result[e]

                        E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                        E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                        E_UB_group += E_UB_county_temp * coe_temp[g * N_e + e]
                        E_LB_group += E_LB_county_temp * coe_temp[g * N_e + e]
                        ell_UB_group += ell_UB_temp * coe_temp[g * N_e + e]

                    E_UB_rec[i][ratio_pubCharger][j] += E_UB_group.reshape(-1) * 56.5 / 1000
                    E_LB_rec[i][ratio_pubCharger][j] += E_LB_group.reshape(-1) * 56.5 / 1000
                    ell_UB_rec[i][ratio_pubCharger][j] += ell_UB_group.reshape(-1) * 56.5 / 1000

                    gap_E = (E_UB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t) - 
                             E_LB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t)).sum(axis = 0)
                    print (f"group={g}, obj={gap_E[-1]}, time={time.time()-start}")



            '''solve the problem for the last group'''
            model_last.pene_para = pene_value
            for n in range(N_n):
                model_last.ell_capacity[n] = capacity_value[n] * groupShare[groupNum, n]

            for e in range(N_e_last):
                # model_last.omega_para[e] = omega_value[groupNum * N_e + e]
                # model_last.q[e] = q[groupNum * N_e + e]
                model_last.coe_E[e] = pene_value * q[groupNum * N_e + e]
                model_last.omega[e] = omega_value[groupNum * N_e + e]
                for t in range(N_t):
                    model_last.power_UB_para[e, t] = power_UB_para_value[groupNum * N_e + e, t]
                    model_last.demand_cum[e, t] = demand_cum[groupNum * N_e + e, t]
                    model_last.coe_county0[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[31][groupNum * N_e + e, t]
                    model_last.coe_county1[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[43][groupNum * N_e + e, t]
                    model_last.coe_county2[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[89][groupNum * N_e + e, t]
                    model_last.coe_county3[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[97][groupNum * N_e + e, t]
                    model_last.coe_county4[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[111][groupNum * N_e + e, t]
                    model_last.coe_county5[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[197][groupNum * N_e + e, t]
                    model_last.coe_county6[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[93][groupNum * N_e + e, t]

            results_last = solver.solve(model_last)

            if str(results_last['Solver'][0]['Termination condition']) == 'optimal':
                y_UB_result = np.array(model_last.y_e_UB[:, :]()).reshape(N_e_last, N_t)
                y_LB_result= np.array(model_last.y_e_LB[:, :]()).reshape(N_e_last, N_t)
                ell_UB_result = np.array(model_last.y_capacity[:, :]()).reshape(N_e_last, N_t)


                E_UB_group = np.zeros((N_n, N_t))
                E_LB_group = np.zeros((N_n, N_t))
                ell_UB_group = np.zeros((N_n, N_t))
                for e in range(N_e_last):
                    E_UB_county_temp = np.zeros((N_n, N_t))
                    E_LB_county_temp = np.zeros((N_n, N_t))
                    ell_UB_temp = np.zeros((N_n, N_t))
                    for n in range(N_n):
                        countyID = countyID_list[n]
                        E_UB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * y_UB_result[e] * 0.25
                        E_LB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * y_LB_result[e] * 0.25
                        ell_UB_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * ell_UB_result[e]

                    E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                    E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                    E_UB_group += E_UB_county_temp * coe_temp[groupNum * N_e + e]
                    E_LB_group += E_LB_county_temp * coe_temp[groupNum * N_e + e]
                    ell_UB_group += ell_UB_temp * coe_temp[groupNum * N_e + e]

                E_UB_rec[i][ratio_pubCharger][j] += E_UB_group.reshape(-1) * 56.5 / 1000
                E_LB_rec[i][ratio_pubCharger][j] += E_LB_group.reshape(-1) * 56.5 / 1000
                ell_UB_rec[i][ratio_pubCharger][j] += ell_UB_group.reshape(-1) * 56.5 / 1000

                gap_E_last = (E_UB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t) - 
                              E_LB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t)).sum(axis = 0)
                print(f"ratio={ratio_pubCharger}, scale={capacity_scale[j]}," \
                       f"pene={pene_value}, obj={gap_E_last[-1]}, time={time.time()-start_all}")
            else:
                print(f"ratio={ratio_pubCharger}, scale={capacity_scale[j]}, pene={pene_value}, infeasible")

            break # only one scale
        break # only one ratio
    break   # only one pene
                

        

start: EV_pene=0.5, ratio=0.1, scale=1.0
group=0, obj=24.344360447338534, time=45.02407670021057
group=1, obj=40.7342887239632, time=47.036826848983765
group=2, obj=57.97297367592017, time=43.803849935531616
group=3, obj=72.7206792900136, time=44.24791693687439
group=4, obj=97.12793351973286, time=49.731224060058594
group=5, obj=144.6047432672651, time=47.81956195831299
group=6, obj=179.0357597354829, time=47.51942777633667
group=7, obj=193.83897180769986, time=56.65703511238098
group=8, obj=218.81881630161035, time=52.785571813583374
group=9, obj=222.9594151820957, time=58.82590913772583
group=10, obj=232.89272852046037, time=48.42407202720642
group=11, obj=405.09442936560066, time=21.898343086242676
group=12, obj=488.6457759043014, time=25.734035968780518
group=13, obj=567.247854775129, time=20.99982190132141
group=14, obj=640.0250634080093, time=20.605743169784546
group=15, obj=699.982922057352, time=20.52567219734192
group=16, obj=757.9439799137151, time=20.817035913467407
group=17

In [ ]:
import os
filePath = 'Results/Chicago2017_RealCapacity/AggregateModel_pubCharger'


for i in range(peneNum):
    pene_value = float(peneList[i])
    path_temp = filePath + f'/pene={pene_value:.1f}'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true 
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
    if not isExists:
        os.makedirs(path_temp)  

    for ratio_pubCharger in ratio_pubCharger_list:
        for j in range(capacityNum):
            """save the energy bounds"""
            E_UB_sparse = sparse.csr_matrix(E_UB_rec[i][ratio_pubCharger][j])
            E_LB_sparse = sparse.csr_matrix(E_LB_rec[i][ratio_pubCharger][j])
            sparse.save_npz(f"{path_temp}/E_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", E_UB_sparse)
            sparse.save_npz(f"{path_temp}/E_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", E_LB_sparse)
            print (f"pene={peneList[i]}, ratio={ratio_pubCharger:.1f}, scale={capacity_scale[j]}, saved")
            break
        break
    break


pene=0.5, ratio=0.1, scale=1.0, saved
pene=0.5, ratio=0.2, scale=1.0, saved
pene=0.5, ratio=0.3, scale=1.0, saved


In [ ]:
capacity_county_MW = {}
capacity_county_withoutHC = np.zeros((N_n, N_t))
for i in range(peneNum):
    pene_value = peneList[i]
    capacity_county_MW[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        capacity_county_MW[i][ratio_pubCharger] = {}
        p_max_total = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()[feasibleTrip]
        for j in range(capacityNum):
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, scale={capacity_scale[j]}")
            capacity_county_MW[i][ratio_pubCharger][j] = np.zeros((N_n, N_t))
            for n in range(N_n):
                for t in range(N_t):
                    county_key = countyIndices[n]
                    capacity_county_withoutHC[n] = (p_max_total* boolIndex[county_key]).T @ (q * pene_value) * 56.5/1000
                    capacity_county_MW[i][ratio_pubCharger][j][n] = np.minimum((p_max_total * boolIndex[county_key]).T @ (q * pene_value) * 56.5/1000, capacity_county[n] * capacity_scale[j] * 56.5/1000)
            break # only one scale
        break # only one ratio
    break   # only one pene

start: EV_pene=0.05, ratio=0.4, scale=1


In [15]:
import os
filePath = 'Results/Chicago2017_RealCapacity/AggregateModel_realPene'


for i in range(peneNum):
    pene_value = float(peneList[i])
    path_temp = filePath + f'/pene={pene_value:.3f}'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true 
    if not isExists:
        os.makedirs(path_temp)  

    for ratio_pubCharger in ratio_pubCharger_list:
        for j in range(capacityNum):
            """save the updated power bound"""
            ell_UB_sparse = sparse.csr_matrix(capacity_county_MW[i][ratio_pubCharger][j].reshape(-1))
            sparse.save_npz(f"{path_temp}/ell_UB_new_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]:.1f}.npz", ell_UB_sparse)
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, scale={capacity_scale[j]}.npz")
            break
        break
    break

start: EV_pene=0.05, ratio=0.4, scale=1.npz
